# Trabalho Prático 1

## Introdução

A resolução deste trabalho prático tem como objetivo aprofundar o conhecimento na implementação de sessões cifradas e seguras num ambiente entre um emissor e um recetor.
Os objetivos principais deste trabalho prático são:

- A construção de uma sessão de comunicação síncrona entre um emissor e um recetor.
- A utilização do protocolo de **Diffie_Hellman** com verificação de chave e autenticação dos agentes na sessão anteriormente referida.
- Utilização do **DSA** para a autenticação dos agentes.
- Utilização do **TAES** como cifra simétrica, com autenticação do criptograma em cada superbloco.
- Finalmente, o último objetivo passa por implementar novamente o cenário anterior mas, desta feita, utilizando o **ECDH** (Elliptic Curve Diffie-Hellman) em vez do **DH** e o **ECDSA** (Elliptic Curve Digital Signature Algorithm) em vez do **DSA**.

Este relatório está escrito de forma a que o texto escrito entre o código desenvolvido seja suficientemente explicativo do que está a ser implementado, o que, na nossa opinião, permite uma leitura e compreensão facilitada do mesmo.

## Sessão Síncrona com _Diffie-Hellman_

### Imports

Esta secção tem como objetivo importar os módulos **Python** necessários para desenvolver a sessão síncrona entre os agentes, utilizando o protocolo **DH** e o **DSA**.

In [1]:
import os
from multiprocessing import Process,Pipe
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
from cryptography.exceptions import InvalidSignature
from cryptography.hazmat.primitives.asymmetric import dsa
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

### Definição da classe de multiprocessamento

Esta secção tem o propósito de definir a classe de multiprocessamento, que permite uma comunicação bidireccional com o _Emitter_ e o _Receiver_, sendo estes dois processos criados e implementados pela **API** [**multiprocessing**](https://docs.python.org/2/library/multiprocessing.html).

In [2]:
class BiConnection(object):
    def __init__(self,left,right):
        left_side,right_side = Pipe()
        self.timeout = None
        self.left_process = Process(target=left,args=(left_side,))
        self.right_process = Process(target=right,args=(right_side,))
        self.left = lambda : left(left_side)
        self.right = lambda : right(right_side)
    
    def auto(self,proc=None):
        if proc == None:
            self.left_process.start()
            self.right_process.start()
            self.left_process.join(self.timeout)
            self.right_process.join(self.timeout)
        else:
            proc.start()
            proc.join()
    
    def manual(self): 
        self.left()
        self.right()

### Geração dos parâmetros _Diffie-Hellman_

Esta secção tem o objetivo de gerar os parâmetros necessários para a geração de chaves **DH** e **DSA** por parte de ambos os agentes.

In [ ]:
parameters_dh = dh.generate_parameters(generator=2,key_size=3072,backend=default_backend())

### Geração das chaves permanentes dos agentes

Esta secção tem como objetivo gerar as chaves permanentes dos agentes envolvidos na comunicação, tendo em conta que estas são as que são utilizadas na troca de chaves utilizando o protocolo **Diffie-Hellman** e na assinatura digital de mensagens.

In [5]:
# Geração das chaves do Emissor
emitter_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
emitter_dsa_pk = emitter_dsa_sk.public_key() #chave pública DSA

# Geração das chaves do Recetor
receiver_dsa_sk = dsa.generate_private_key(3072,default_backend()) #chave privada DSA
receiver_dsa_pk = receiver_dsa_sk.public_key() #chave pública DSA

### Definição do Emissor e do Recetor

Nesta secção está definido o comportamento de cada um dos agentes ao utilizar esta sessão síncrona segura de troca de informação. Na definição destes dois agentes, é assumido que as chaves públicas **DSA** utilizadas na autenticação dos agentes já são conhecidas por ambos. As chaves **DH** são geradas por sessão de modo a acordar uma chave temporária para que seja possível cifrar e decifrar uma mensagem utilizando uma primitiva simétrica.

In [6]:
def Emissor(connection):
    emitter_dh_sk = parameters_dh.generate_private_key()
    emitter_dh_pk = emitter_dh_sk.public_key()
    pub = emitter_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = emitter_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk': pub, 'sig': signature}
    connection.send(message)
    receiver_first = connection.recv()
    receiver_dh_public_key = serialization.load_pem_public_key(receiver_first['pk'],default_backend())
    receiver_dsa_pk.verify(receiver_first['sig'],receiver_first['pk'],hashes.SHA256())
    print("OK")
    

In [7]:
def Recetor(connection):
    emitter_first = connection.recv()
    receiver_dh_sk = parameters_dh.generate_private_key()
    receiver_dh_pk = receiver_dh_sk.public_key()
    emitter_dh_public_key = serialization.load_pem_public_key(emitter_first['pk'],default_backend())
    emitter_dsa_pk.verify(emitter_first['sig'],emitter_first['pk'],hashes.SHA256())
    pub = receiver_dh_pk.public_bytes(
        encoding = serialization.Encoding.PEM,
        format = serialization.PublicFormat.SubjectPublicKeyInfo)
    signature = receiver_dsa_sk.sign(pub,hashes.SHA256())
    message = {'pk':pub,'sig':signature}
    connection.send(message)
    print("OK")
    

In [ ]:
BiConnection(Emissor,Recetor).auto()

Process Process-1:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-a5e5f453a72f>", line 2, in Emissor
    emitter_dh_sk = parameters_dh.generate_private_key()
NameError: name 'parameters_dh' is not defined
